<a id='logbook'></a>
# Logbook

In [ ]:
# %load ../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score



## Nomenclature
* **Ikeda** refer to the original ikeda method (require strip calculations and Lewis sections).
* **Simplified Ikeda** or **SI** refer to the simplified Ikeda method (explicit formula).

[Logbook](#logbook)
## 2020-09-22
I've been working with adding additional material to extend the paper for the [SAOC](https://www.tandfonline.com/toc/tsos20/current).

### Ikeda for one ship
I have implemented the original ikeda method and made calculations for one of the ships in the database: [01.01_ikeda_one_ship](06_ikeda/01.01_ikeda_one_ship.ipynb). I developed a python wrapper for SSPA strip calculation tool *ScoresII* here: [pyscores2](https://github.com/martinlarsalbert/pyscores2) and calculated the wave roll damping $B_W$.
The results were compared with the Simplified Ikeda (SI).
* It was found that the $B_W$ was quite different. It was however hard to say which one was the better. 

* It was also found that the speed dependecy of the eddy component $B_E$ was not implemented as in <cite data-cite="7505983/ZVF3F5V5"></cite>.

* It was also found that the $B_{BK}$ contribution was quite different: [Ikeda vs. SI](06_ikeda/01.01_ikeda_one_ship.ipynb#ikeda_vs_si). 

* For this ship $B_{BK}$ calculated with original ikeda implementation gave better results:
[Ikeda vs. SI2](06_ikeda/01.01_ikeda_one_ship.ipynb#ikeda_vs_si2).

I made a new comparison between SI and model test results where the influence of findings above were investigated ([07.2_si_compare](04_simplified_ikeda/07.2_si_compare.ipynb)). The $B_E$ speed dependency gave an improvement. Switching the $B_{BK}$ also gave an improved accuracy with maximum accuracy with fixed bilge radius $ \frac{R}{beam}=0.08$ according to:[scores](04_simplified_ikeda/07.2_si_compare.ipynb#scores). 

### Validation 
I also made a validation study to reproduce results from Carl-Johan for the *S175* ship: [Ikeda S175](06_ikeda/01.02_ikeda_S175.ipynb).


## 2020-09-23

### Recalculating $B_E$
[Sectional Lewis coefficients](06_ikeda/01.01_ikeda_one_ship.ipynb#lewis_coefficients) have been calculated. These are used to calculate the eddy damping: $B_E$ [here](06_ikeda/01.01_ikeda_one_ship.ipynb#eddy). This is also a translation of Carl-Johans matlab code which seams to be an implementation according to <cite data-cite="7505983/RWXBIIJJ"></cite> with some exceptions where changes according to <cite data-cite="7505983/RC2WJ4RI"></cite>. It was a bit unclear if the $\overline{OG}$ should be possitive uppward (Journe) or into the water (Ikeda). It was also unclear if Journe or Ikeda was the best, this should be investigated!

<cite data-cite="7505983/RWXBIIJJ"></cite> also proposes these formulas to estimate the bilge radius:
$$
R=\left\{\begin{array}{ll}
2 a \sqrt{\frac{a_{0}(\sigma-1)}{\pi-4}} & (R<d, R<B / 2) \\
a & \left(H_{0} \geq 1, R / d>1\right) \\
B / 2 & \left(H_{0} \leq 1, R / d>H_{0}\right)
\end{array}\right.
$$

## 2020-09-24
* I found that there might be a difference in the speed dependency between Ikeda and Journe:

<cite data-cite="7505983/ZVF3F5V5"></cite>
$$
\begin{aligned}
\frac{B_{W}}{B_{k 0}} &=0.5\left[\left(\left(A_{2}+1\right)+\left(A_{2}-1\right) \tanh (20 \tau-b)\right\}\right.\\
&\left.+\left(2 A_{1}-A_{2}-1\right) \exp \left\{-150(\tau-0.25)^{2}\right\}\right]
\end{aligned}
$$


<cite data-cite="7505983/RC2WJ4RI"></cite>
$$
B_{44 S}=B_{44} \cdot\left\{0.5 \cdot\left(\begin{array}{l}
A_{2}+1+\left(A_{2}-1\right) \cdot \tanh [20 \cdot(\Omega-0.3)] \\
+\left(2 \cdot A_{1}-A_{2}-1\right) \cdot e^{-150(\omega-0.25)^{2}}
\end{array}\right)-1.0\right\}
$$

The difference is the $b$ coefficient which Journee has set to $20*0.3$. <cite data-cite="7505983/RWXBIIJJ"></cite> call this $b$ the extinction coefficient.

* I found that the speed dependency correction for $B_W$ is not $1$ when the ship speed is $0$ which is quite strange. Is there an implementation error there?

Here is the current implementation (which is clearly wrong according to the plot below, since it is not 1 at V=0). Have I, Carl-Johan or Journée made a misstaker here???

In [ ]:
from numpy import exp, tanh
def B_W_speed_correction_factor(w, V, d, g=9.81):
    """
        Wave damping speed correction
        Parameters
        ----------
        w
            "omega" frequency of motion [rad/s]
        V
            ship speed [m/s]
        d
            ship draught [m]
        g
            gravity

        Returns
        -------
        Bw_div_Bw0
            Bw_div_Bw0 = B_W/B_W0

        """
    OMEGA = w * V / g
    zeta_d = w ** 2 * d / g
    A1 = 1 + zeta_d ** (-1.2) * exp(-2 * zeta_d)
    A2 = 0.5 + zeta_d ** (-1) * exp(-2 * zeta_d)

    Bw_div_Bw0 = 0.5 * (
            ((A1 + 1) + (A2 - 1) * tanh(20 * (OMEGA - 0.3))) + (2 * A1 - A2 - 1) * exp(-150 * (OMEGA - 0.25) ** 2))

    return Bw_div_Bw0

In [ ]:
w = 0.2
lpp = 175
omega_hat = 0.719
beam = 25.40
d = 9.5
g=9.81
w = lambdas.omega(beam=beam, g=g, omega_hat=omega_hat)
Fn = np.linspace(0, 0.3, 100)
V = Fn*np.sqrt(lpp*g)


factor = B_W_speed_correction_factor(w=w, V=V, d=d)

fig,ax=plt.subplots()
ax.plot(Fn,factor)
ax.grid(True)
ax.set_ylabel(r'$\frac{B_W}{B_{W0}}$')
ax.set_xlabel('Ship speed Fn [-]');
ax.set_title('$B_W$ Speed dependency factor');

This plot should look something as:
<img src="06_ikeda/himeno_B_W_div_BW0.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />


I would say that the Journee equation is even worse:

In [ ]:
from rolldecayestimators.ikeda_speed import B_W_speed_correction_factor_journee

factor = B_W_speed_correction_factor_journee(w=w, V=V, d=d)

fig,ax=plt.subplots()
ax.plot(Fn,factor)
ax.grid(True)
ax.set_ylabel(r'$\frac{B_W}{B_{W0}}$')
ax.set_xlabel('Ship speed Fn [-]');
ax.set_title('$B_W$ Speed dependency factor');

I went back to the original Ikeda paper <cite data-cite="7505983/ZVF3F5V5"></cite> and this implementation is in fact much more reasonable. The Himeno equation is the same (but the value of $b$) is not written out (it is $20*0.3$ in Ikeda). I found a typo in Carl-Johans implementation which was the root cause of this error. (But Journée is simply wrong still).

In [ ]:
def B_W_speed_correction_factor_ikeda(w, V, d, b=20*0.3, g=9.81):
    """
        Wave damping speed correction
        Parameters
        ----------
        w
            "omega" frequency of motion [rad/s]
        V
            ship speed [m/s]
        d
            ship draught [m]
        g
            gravity

        Returns
        -------
        Bw_div_Bw0
            Bw_div_Bw0 = B_W/B_W0

        """
    tau=w*V/g
    zeta_d=w**2*d/g
    A1=1+zeta_d**(-1.2)*exp(-2*zeta_d);
    A2=0.5+zeta_d**(-1)*exp(-2*zeta_d);

    
    Bw_div_Bw0=0.5*((A2+1)+(A2-1)*tanh(20*tau-b)
                    +(2*A1-A2-1)*exp(-150*(tau-0.25)**2))

    return Bw_div_Bw0

In [ ]:
factor = B_W_speed_correction_factor_ikeda(w=w, V=V, d=d)

fig,ax=plt.subplots()
ax.plot(Fn,factor)
ax.grid(True)
ax.set_ylabel(r'$\frac{B_W}{B_{W0}}$')
ax.set_xlabel('Ship speed Fn [-]');
ax.set_title('$B_W$ Speed dependency factor');

Should the speed dependence factor be modified, here is an example where $b$ is varied:

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,4)
    
for b in np.linspace(1,8,8):
    factor = B_W_speed_correction_factor_ikeda(w=w, V=V, d=d, b=b)

    ax.plot(Fn,factor, label='b=%0.1f'%b)

ax.grid(True)
ax.set_ylabel(r'$\frac{B_W}{B_{W0}}$')
ax.set_xlabel('Ship speed Fn [-]');
ax.set_title('$B_W$ Speed dependency factor');
ax.set_ylim(1,3)
ax.legend();
    

In [ ]:
def f(b=6.0,omega_hat=0.719):
    
    w = lambdas.omega(beam=beam, g=g, omega_hat=omega_hat)
    factor = B_W_speed_correction_factor_ikeda(w=w, V=V, d=d, b=b)

    fig,ax=plt.subplots()
    fig.set_size_inches(10,4)
    ax.plot(Fn,factor)
    ax.grid(True)
    ax.set_ylabel(r'$\frac{B_W}{B_{W0}}$')
    ax.set_xlabel('Ship speed Fn [-]');
    ax.set_title('$B_W$ Speed dependency factor');
    ax.set_ylim(1,4)

In [ ]:
from ipywidgets import interactive

interactive_plot = interactive(f, b=(0.1,8,0.5),omega_hat=(0.2,1,0.1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

## 2020-09-25
* Rerun the [04.2_maa_mdl_db_build.ipynb](rolldecay/02_roll_damping_DB/04.2_maa_mdl_db_build.ipynb).
  * With fixed $B_W$ speed dependency
  * Alternative bilge keel R/B=0.08
* Rerun the cross validation and got ([quite much better score with SI](05_new_method/05.1_maa_new_method_and_correction.ipynb#score))

* Made a new sensituvity study of the SI with some interesting plots: ([07.3_si_sensitivity2.ipynb](04_simplified_ikeda/07.3_si_sensitivity2.ipynb)).

## 2020-09-28
* Improved the plots in the ([07.3_si_sensitivity2.ipynb](04_simplified_ikeda/07.3_si_sensitivity2.ipynb)). It seems unlikely that the $\hat{\omega}$ and $F_n$ dependencies are correct in the SI.

## Conclusions
* How the parts in Ikeda's method should be implemented is not obvious
    * The old papers are difficult to interpret (missing nomenclature etc.)
    * Typos in old papers
    * There exist various variations of the implementations (ex: Ikeda, Journée etc.) giving various "flavors" of the method.
* Validating the "flavors" of Ikeda method with model tests to select the best one.
* Found a implementation error in the $B_W$ speed dependency. Which has now been corrected as described above. This error does not seem to have a huge influence on the overall accuracy, but this will be further investigated.

## References
<div class="cite2c-biblio"></div>